In [1]:
# Kütüphaneleri yüklüyoruz.

import cv2
import numpy as np

In [2]:
# Tracking işlemi yapıldığı için RAM dolup kod kapanmasın diye bunu çalıştırıyoruz.
# Bazı donanım ya da kütüphane sürümlerine gerekli olmayabilir.

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [4]:
# Burada modeli yüklüyoruz.
from ultralytics import YOLO
model = YOLO("yolov8l.pt")

In [6]:
font = cv2.FONT_HERSHEY_DUPLEX
kamera = cv2.VideoCapture("video.mp4")


# Burada geçişin olduğu yeri ben belirledim. Yani bir çizgi seçtim. Bu çizgiy geçerse geçti kabul ediliyor.
# Bu çizgiyi kendi videonuza göre uygun şekilde belirleyebilirsiniz. 
# Burada yolun sağında giden arabalar solunda ise gelen arabalar var. Bu yüzden iki tane bölge var.
# Bu bölgelerden yolun sağında olan çizginin üstünde, yolun solunda olan çizginin altında yer alıyor.
# Bu bölgelerin amacı aracın çizgiyi geçip geçmediğini bulmak. Eğer bu bölgelere girişmişse çizgi geçilmiştir.
# Sol taraftaki bölgeye giren gelen sağ taraftaki bölgeye giren araç ise giden olarak kabul ediliyor.

# Çizginin koordinatları (100,550), (1100,550)
# Bölgeler de buna uygun


# Burası yolun sol tarafı için
region1=np.array([(100,550),(550,550),(550,570),(100,570)])
region1 = region1.reshape((-1,1,2))


# Burası yolun sağ tarafı için
region2=np.array([(700,550),(1100,550),(1100,530),(700,530)])
region2 = region2.reshape((-1,1,2))



# Sol taraftan geçen araçların id'si buna eklenecek.
toplam_sol=set()
# Sağ taraftan geçen araçların id'si buna eklenecek.
toplam_sag=set()


while True:

    ret, frame = kamera.read()
    
   
    
    if not ret:
        break
        
 

    rgb_img=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model.track(rgb_img, persist=True, verbose=False)
    labels=results[0].names
    
    # Çizgiyi çizdiriyoruz. 
    cv2.line(frame, (100,550), (1100,550), (255,255,255), 5)
    # Ben bölgeleri çizdirmedim. Siz isterseniz çizdirebilirsiniz.
    #cv2.polylines(frame,[region1],True,(0,0,255),2)
    #cv2.polylines(frame,[region2],True,(0,0,255),2)
    
    
    for i in range(len(results[0].boxes)):
        x1,y1,x2,y2=results[0].boxes.xyxy[i]
        score=results[0].boxes.conf[i]
        cls=results[0].boxes.cls[i]
        ids=results[0].boxes.id[i]
        x1,y1,x2,y2,score,cls,ids=int(x1),int(y1),int(x2),int(y2),float(score),int(cls),int(ids)
        
        name=labels[cls]
        # burada 0.5' lik bir threshold uyguladık. Tespit edilme değeri daha küçükse değerlendirmeye almıyoruz. 
        if score<0.5:
            continue
        # Araçların  ortasının konumu buluyoruz.
        cx=int(x1/2+x2/2)
        cy=int(y1/2+y2/2)
        
        
        

        
     
        # Aracın orta nnoktası sol bölgeye girip girmediği kontrol ediliyor. Girip girmemesine göre bir değer döner
        # Eğer girmişse bu değer pozitiftir.
        inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
        
        # Eğer aracın orta noktası sol bölgeye girmişse if bloğunun içine girilir.
        if inside_region1>0:
            toplam_sol.add(ids)
            # Araçlar geçtiği zaman çizginin rengini değiştiriyorum.
            # Ayrıca sadece bu bölgelere girilidği zaman araç yeşil bir kutu içine alınıyor.
            # Ayrıca orta noktası ekranda gösteriliyor.
            cv2.circle(frame,(cx,cy),15,(255,0,0),-1)
            cv2.line(frame, (100,550), (1100,550), (0,255,255), 5)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),5)
            
        # Aracın orta noktası sağ bölgeye girip girmediği kontrol ediliyor. Girip girmemesine göre bir değer döner
        # Eğer girmişse bu değer pozitiftir.
        
        inside_region2=cv2.pointPolygonTest(region2,(cx,cy),False)
        
        # Eğer aracın orta noktası sağ bölgeye girmişse if bloğunun içine girilir.
        if inside_region2>0:
            toplam_sag.add(ids)
            
            cv2.circle(frame,(cx,cy),15,(255,0,0),-1)
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),5)
            cv2.line(frame, (100,550), (1100,550), (0,255,255), 5)
            
    # Gelen  ve giden araç sayısını ekranda göstermek için bunları string formatına çevirdik
    # Burada Gelen  ve giden araç sayısı bunlarda kaç tane eleman varsa o kadardır.
    # Burada set kullanma sebebimiz, bunlarda aynı elemanın sadece bir kere bulunmasıyla ilgili.
    # Eğer liste olsaydı aynı eleman birden fazla olabilirdi

    toplam_sol_str='GELEN: '+str(len(toplam_sol))
    toplam_sag_str='GIDEN: '+str(len(toplam_sag))
    
    # Ekrana yazdırırken güzel dursun diye görüntünün sol üst ve sağ üst köşesine arka plan rengi ayarladık.
    frame[0:80,0:330]=(102,0,153)
    frame[0:80,950:1280]=(102,0,153)
    # Ekranda gösteriyoruz.
    cv2.putText(frame, toplam_sol_str,(0, 60), font, 2, (255,255,255), 2)
    cv2.putText(frame, toplam_sag_str,(950, 60), font, 2, (255,255,255), 2)
            
        

    
  
    
    
    cv2.imshow("frame", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

kamera.release()
cv2.destroyAllWindows()